In [63]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

In [64]:
with open('./input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

unique_chars = sorted(list(set(text)))
vocab_size = len(unique_chars)

stoi = { ch:i for i,ch in enumerate(unique_chars) }
itos = { i:ch for i,ch in enumerate(unique_chars) }

tokens = torch.Tensor([stoi[x] for x in list(text)][:5000])

train_split = 0.9
n = int(train_split * len(tokens))
train, test = tokens[:n], tokens[n:]

In [65]:
class TokensDataset(Dataset):

    def __init__(self, series, seq_length=10, device="cpu"):
        self.series = series
        self.seq_length = seq_length

    def __len__(self):
        return len(self.series) - self.seq_length - 1

    def __getitem__(self, idx):
        X = self.series[idx:idx+self.seq_length].reshape(self.seq_length, -1).to(device)
        y = self.series[idx+self.seq_length].to(device)

        return X, y

In [66]:
train_dataset = TokensDataset(train, seq_length=10, device=device)
test_dataset = TokensDataset(test, seq_length=10, device=device)
train_data = DataLoader(train_dataset, batch_size=1024, shuffle=True)
test_data = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [67]:
class RNN(nn.Module):

    def __init__(self, input_size, hidden_size):

        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True) # (batch, time, input_size) -> (batch, time, hidden_size)
        self.fc = nn.Sequential(
            nn.ReLU(),
            nn.Linear(in_features=hidden_size, out_features=1), # out_features = 1 to predict the next value
        )

    def forward(self, x):
        output, status = self.rnn(x)
        output = output[:, -1, :] # (batch, time, hidden_size) -> (batch, 1, hidden_size). We take only the final output of the RNN
        output = self.fc(output)

        return output

In [73]:
input_size = 1
hidden_size = 5
learning_rate = 1e-4
num_epochs = 100
# ----

model = RNN(input_size, hidden_size).to(device)

criterion = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [74]:
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    for batch, targets in tqdm(train_data, desc="Batches", leave=False):

        outs = model(batch).reshape(-1)

        optimizer.zero_grad()
        loss = criterion(outs, targets)
        loss.backward()
        optimizer.step()

    print(f"After epoch {epoch}, loss={loss.item()}")

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

After epoch 0, loss=1992.2626953125


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

After epoch 1, loss=1885.878173828125


...

After epoch 97, loss=1796.647216796875


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

After epoch 98, loss=1938.6146240234375


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

After epoch 99, loss=1990.7935791015625


In [75]:
import torch.nn.functional as F

def predict(model, start_text, length=100, temperature=1.0):
    model.eval()
    
    # Convert input text to tensor
    input_seq = torch.tensor([stoi[ch] for ch in start_text], dtype=torch.float32).reshape(1, -1, 1).to(device)

    generated = start_text
    for _ in range(length):
        with torch.no_grad():
            output = model(input_seq)  # Get model output
            probs = F.softmax(output / temperature, dim=-1)  # Apply temperature scaling
            
            # Sample from probability distribution
            next_char_idx = torch.multinomial(probs, num_samples=1).item()

        generated += itos[next_char_idx]
        
        # Update input sequence
        next_input = torch.tensor([[next_char_idx]], dtype=torch.float32, device=device).reshape(1, 1, 1)
        input_seq = torch.cat((input_seq[:, 1:, :], next_input), dim=1)  # Keep seq length fixed

    return generated


In [76]:
start_text = "hello worl"
generated_text = predict(model, start_text, length=50, temperature=0.8)
print("Generated Text:", generated_text)


Generated Text: hello worl


















































